# Way too advanced for intro

# Functions as Objects

<style>
section.present > section.present { 
    max-height: 90%; 
    overflow-y: scroll;
}
</style>

<small><a href="https://colab.research.google.com/github/brandeis-jdelfino/cosi-10a/blob/main/lectures/notebooks/14_functions_as_objects.ipynb">Link to interactive slides on Google Colab</a></small>

## Exercise

Sort the words in a string alphabetically

In [ ]:
value = "This note is about Bill Murray, a famous American actor."
words = value.split()
words.sort()
print(' '.join(words))

Hmm... case matters.

We could just lowercase the whole string before splitting. But what if we want to preserve the original case in our resulting string?

## Telling `sort` how to sort

By default, `sort()` orders the values by... the values.

We can tell it to order them by something else, using the `key` parameter.

The `key` parameter should be a **function** that returns the sort key for a given value.

In [ ]:
def make_lower(value):
    return value.lower()

value = "This note is about Bill Murray, a famous American actor."
words = value.split()

words.sort(key=make_lower)
print(' '.join(words))

Wait, what? Did we just pass a function as a parameter?

Yes, yes we did.

# Functions are Objects

Python has **first class functions**. 

This means that functions are objects, just like other types: `int`, `str`, `list`, `dict`, `set`, etc.

They can be passed as arguments to other functions.

They can be assigned to variables.

They can be returned by other functions.

They can be stored in lists, dictionaries, and sets.

For another example, we'll sort the words by length:

In [ ]:
def get_len(value):
    return len(value)

value = "This note is about Bill Murray, a famous American actor."
words = value.split()

words.sort(key=get_len)
print(' '.join(words))

But wait, `len` is a function too. We don't need to make our own wrapper, `get_len`. We can pass the function `len` directly!

In [ ]:
value = "This note is about Bill Murray, a famous American actor."
words = value.split()

words.sort(key=len)
print(' '.join(words))

Turns out, we can also shorten our first example. Here's the original code:

In [ ]:
def make_lower(value):
    return value.lower()

value = "This note is about Bill Murray, a famous American actor."
words = value.split()

words.sort(key=make_lower)
print(' '.join(words))

We can actually rewrite this as:

In [ ]:
value = "This note is about Bill Murray, a famous American actor."
words = value.split()

words.sort(key=str.lower)
print(' '.join(words))

What is `str.lower`? Let's dig in to understand.

## Review: calling methods

In [ ]:
class Tree:
    def __init__(self):
        self.height = 1

    def grow(self):
        self.height += 1

In [ ]:
t = Tree()
t.grow()
t.grow()
print(t.height)

Notice that `grow()` takes the `self` parameter, but when we call `t.grow()`, we don't pass any parameters.

When we call `t.grow()`, `t` "magically" gets passed as `self` to the `grow()` method.

Here, we explicitly pass `t` for the `self` parameter:

In [ ]:
t = Tree()
Tree.grow(t)
Tree.grow(t)
print(t.height)

This is equivalent to the previous code.

Note that we call `Tree.grow` here instead of `t.grow`. 

You wouldn't normally call a method this way, but it is completely valid.

## Back to `str.lower`

`str.lower` is a method on the `str` class, just like `grow` is a method on the `Tree` class. 

`lower`'s definition might look like this:

In [ ]:
class str:
    # ...
    def lower(self):
        # ...

This means that `"SOME STRING".lower()` is equivalent to `str.lower("SOME STRING")`.

So, when we pass `str.lower` as a function here, `sort` will call it on each value in the list in order to get the sort key: 

`str.lower(value)`  
which is equivalent to:  
`value.lower()`

In [ ]:
value = "This note is about Bill Murray, a famous American actor."
words = value.split()

words.sort(key=str.lower)
print(' '.join(words))

## Some more examples

What if we want to sort all words that start with `M` or `m` first, then alphabetically after that for every non-M word?

In [ ]:
def m_first(value):
    # upper case sorts first, so if our word starts with "M" or "m", use upper case for the sort key
    if value[0].lower() == 'm':
        return value.upper()
    else:
        return value.lower()
    
value = "This note is about Bill Murray, a famous American actor."
words = value.split()

words.sort(key=m_first)
print(' '.join(words))    

## Lambdas

We can even shorten this one, using something called a **lambda**.

A lambda is an "anonymous function" - a function with no name.

In [ ]:
value = "This note is about Bill Murray, a famous American actor."
words = value.split()

words.sort(key=lambda x: value.upper() if x[0].lower() == 'm' else value.lower())
print(' '.join(words)) 

## Lambda syntax

`lambda <parameter list>: <expression>`

Lambdas can take any number of parameters, but can only consist of a single expression.

They can be useful, but also can easily make code hard to read. Often, writing out a full function is more clear.

# Exercise: Write a class that handles secret messages

It should allow a user to store messages and retrieve messages. 

First, we'll define some encode/decode functions. We'll use a "Caesar cipher", which just shifts each character forward in the alphabet.

In [ ]:
def caesar_encode(value):
    new_val = ""
    for c in value:
        new_val = new_val + chr(ord(c) + 7)
    return new_val

def caesar_decode(value):
    new_val = ""
    for c in value:
        new_val = new_val + chr(ord(c) - 7)
    return new_val

In [ ]:
caesar_encode("Hi mom!")

In [ ]:
caesar_decode("Op'tvt(")

Now we'll make a Message repository that encodes messages when storing, and decodes them when retrieving.

In [ ]:
class MessageRepository:
    def __init__(self):
        self.next_id = 0
        self.messages = {}
        
    def get_message(self, message_id):
        if message_id not in self.messages:
            return None
        return caesar_decode(self.messages[message_id])
    
    def store_message(self, message):
        self.messages[self.next_id] = caesar_encode(message)
        self.next_id += 1
        return self.next_id - 1

In [ ]:
repo = MessageRepository()
msg_id = repo.store_message("Hi mom!")

print(repo.messages)
print(repo.get_message(msg_id))

But what if we want to alter the way we encode or decode messages? Or use different methods for different messages?

We could try providing a menu of options... or we could allow the user to provide encode/decode functions!

In [ ]:
class MessageRepository:
    def __init__(self):
        self.next_id = 0
        self.messages = {}
        
    def get_message(self, message_id, decode_fn):
        if message_id not in self.messages:
            return None
        return decode_fn(self.messages[message_id])
    
    def store_message(self, message, encode_fn):
        self.messages[self.next_id] = encode_fn(message)
        self.next_id += 1
        return self.next_id - 1

In [ ]:
repo = MessageRepository()
msg_id = repo.store_message("Hi mom!", caesar_encode)
print(repo.messages)
print(repo.get_message(msg_id, caesar_decode))

You could also imagine a `MessageRepository` that took encode/decode functions on creation, and saved them for later use:

In [ ]:
class MessageRepository:
    def __init__(self, encode_fn, decode_fn):
        self.next_id = 0
        self.messages = {}
        self.encode_fn = encode_fn
        self.decode_fn = decode_fn
        
    def get_message(self, message_id):
        if message_id not in self.messages:
            return None
        return self.decode_fn(self.messages[message_id])
    
    def store_message(self, message):
        self.messages[self.next_id] = self.encode_fn(message)
        self.next_id += 1
        return self.next_id - 1

In [ ]:
repo = MessageRepository(caesar_encode, caesar_decode)
msg_id = repo.store_message("Hi mom!")
print(repo.messages)
print(repo.get_message(msg_id))

## What if we want a Caesar cipher that uses a different offset?

Normally, we'd just add a parameter to `caesar_cipher`. 

In [ ]:
def caesar_encode(value, n):
    new_val = ""
    for c in value:
        new_val = new_val + chr(ord(c) + n)
    return new_val

def caesar_decode(value, n):
    new_val = ""
    for c in value:
        new_val = new_val + chr(ord(c) - n)
    return new_val

In [ ]:
repo = MessageRepository(caesar_encode, caesar_decode)
msg_id = repo.store_message("Hi mom!")
print(repo.messages)
print(repo.get_message(msg_id))

Our class, `MessageRepository`, expects our encode/decode functions to take 1 argument - the value to encode/decode. There's no way to get that extra argument to it without modifying the class.

We could write a different function for each offset we wanted to use... but that's pretty tedious.

Or we could use a lambda:

In [ ]:
def caesar_encode(value, n):
    new_val = ""
    for c in value:
        new_val = new_val + chr(ord(c) + n)
    return new_val

repo = MessageRepository()
msg_id = repo.store_message("Hi mom!", lambda x: caesar_encode(x, 10))
print(repo.messages)
print(repo.get_message(msg_id, lambda x: caesar_encode(x, -10)))

Here, `lambda x: caesar_encode(x, 10)` creates a function that takes one argument, and then calls caesar_encode with that argument and `10` for `n`.

This is perfectly good. 

However, for the sake of exploration, let's use a function... to create a function!

In [ ]:
def make_caesar_shift(n):
    def caesar_shift(value):
        new_val = ""
        for c in value:
            new_val = new_val + chr(ord(c) + n)
        return new_val
    return caesar_shift

Here, `make_caesar_shift` is a function that creates, and returns, another function. 

We can use `make_caesar_shift` to create functions that we store in variables, call, or pass around, just like any other function.

In [ ]:
caesar_encode10 = make_caesar_shift(10)
caesar_decode10 = make_caesar_shift(-10)

encoded = caesar_encode10("Hi mom!")
print(encoded)
print(caesar_decode10(encoded))

Here's an example of using it with `MessageRepository`:

In [ ]:
def make_caesar_shift(n):
    def caesar_shift(value):
        new_val = ""
        for c in value:
            new_val = new_val + chr(ord(c) + n)
        return new_val
    return caesar_shift

repo = MessageRepository(make_caesar_shift(5), make_caesar_shift(-5))
msg_id = repo.store_message("Hi mom!")
print(repo.messages)
print(repo.get_message(msg_id))

## Exercise

Write a function that tracks how long the execution of another function takes.

First, let's see what this would look like without worrying about functions as objects.

In [ ]:
import time

def sum_to_100k():
    total = 0
    for i in range(100000):
        total += i
    return total

start_time = time.time()
sum_to_100k()
end_time = time.time()
print(f"That took {(end_time - start_time)*1000} ms")

Ok, now let's write a timer function that takes another function as an argument:

In [ ]:
def timed(func):
    start_time = time.time()
    func()
    end_time = time.time()
    print(f"That took {(end_time - start_time)*1000} ms")

In [ ]:
timed(sum_to_100k)

But what if we want to time a function that takes arguments?

In [ ]:
def timed(func):
    start_time = time.time()
    func()
    end_time = time.time()
    print(f"That took {(end_time - start_time)*1000} ms")
    
def sum_to(n):
    total = 0
    for i in range(n):
        total += i
    return total

`timed` takes in a function, `func`, which can't take any arguments. This is because `timed` calls `func` with 0 arguments.

One way to use `sum_to(n)` would be to use a lambda to create a function that takes 0 arguments:

In [ ]:
timed(lambda: sum_to(100000))

This is a little awkward.

Luckily, Python gives us 2 special arguments: `*args` and `**kwargs`.

`*args` represents all the positional paramaters, and `**kwargs` represents all the keyword (named) parameters passed to a function. (The `kw` in `kwargs` stands for "keyword".)

In [ ]:
def timed(func, *args, **kwargs):
    start_time = time.time()
    func(*args, **kwargs)
    end_time = time.time()
    print(f"That took {(end_time - start_time)*1000} ms")

In [ ]:
timed(sum_to, 100000)

`timed` now accepts a function, `func`, and any number of additional arguments. All those extra arguments are "passed through" to `func`.

`*args` and `**kwargs` effectively act as wildcard parameters, and let `timed` pass them through without needing to know what they are, or how many of them there are.

In [ ]:
def timed(func, *args, **kwargs):
    start_time = time.time()
    func(*args, **kwargs)
    end_time = time.time()
    print(f"That took {(end_time - start_time)*1000} ms")

In [ ]:
def silly_sum_to(n, a, b, c, d, e, f, g):
    return sum_to(n)

timed(silly_sum_to, 100000, 1, 2, 3, 4, 5, 6, 7)

## Using functions to make functions

Our last version of `timed` ran the function when it was called.

What if, instead, we made it create a new, timed version of our function that we could call later?



Here's a new version of `timed` that returns a function wrapped in a timer. 

This `timed` doesn't run the function directly - it creates a new function and returns it, similar to `make_caesar_shift`.

In [ ]:
def timed(func):
    def wrapped():
        start_time = time.time()    
        func()
        end_time = time.time()
        print(f"That took {(end_time - start_time)*1000} ms")        
    return wrapped

In [ ]:
frankenfunc = timed(sum_to)
frankenfunc(100)
frankenfunc(2000000)

This has the same problem with passing arguments - `*args` and `**kwargs` will work again here.

In [ ]:
def timed(func):
    def wrapped(*args, **kwargs):
        start_time = time.time()    
        func(*args, **kwargs)
        end_time = time.time()
        print(f"That took {(end_time - start_time)*1000} ms")        
    return wrapped

def sum_to(n):
    total = 0
    for i in range(n):
        total += i
    return total

In [ ]:
frankenfunc = timed(sum_to)
frankenfunc(100)
frankenfunc(2000000)

We won't go deeper into `*args` and `**kwargs` in this class. You can do interesting and complicated (often: overcomplicated) things with them. 

If you want to read more about them, here's an [introduction](https://www.programiz.com/python-programming/args-and-kwargs).

The important point for this exercise is that they make it possible for intermediate functions to "pass through" sets of arguments without needing to know what they are, or how many there are.

## Decorators

Python has a feature called "decorators", which is "syntactic sugar" for wrapping a function in another function.

In [ ]:
def timed(func):
    def wrapped(*args, **kwargs):
        start_time = time.time()    
        func(*args, **kwargs)
        end_time = time.time()
        print(f"That took {(end_time - start_time)*1000} ms")        
    return wrapped

@timed
def sum_to(n):
    total = 0
    for i in range(n):
        total += i
    return total

In [ ]:
sum_to(1000000)

## Decorators

Creating a decorator is a fairly advanced Python programming activity.

However, using them is common - many libraries to provide them to use, so you should recognize them, and understand that they are modifying the function they apply to, often in complex, seemingly-magical ways.

## Wrap up

We covered some pretty advanced concepts today. Many of these are beyond the scope of a typical introductory level programming course.

If you don't feel comfortable with the idea that functions are objects, or didn't follow the examples up to (but not including) `make_caesar_cipher`, you should review the beginning of the lecture.

If you felt lost after that, please don't hesitate to reach out with questions if you want to understand... but also don't expect to see these topics on a quiz or in the last problem set.